# 5장. XGBoost 소개

*아래 링크를 통해 이 노트북을 주피터 노트북 뷰어(nbviewer.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.org/github/rickiepark/handson-gb/blob/main/Chapter05/Advanced_XGBoost_Unveiled.ipynb"><img src="https://jupyter.org/assets/share.png" width="60" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/handson-gb/blob/main/Chapter05/Advanced_XGBoost_Unveiled.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

In [1]:
# 노트북이 코랩에서 실행 중인지 체크합니다.
import sys
if 'google.colab' in sys.modules:
    !pip install -q --upgrade xgboost
    !wget -q https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter05/atlas-higgs-challenge-2014-v2.csv.gz

In [2]:
# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
xgb.set_config(verbosity=0)

## XGBoost 모델 만들기

### 붓꽃 데이터셋

In [3]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                  columns=iris['feature_names'] + ['target'])

In [5]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(iris['data'], 
                                                    iris['target'], random_state=2)

#### XGBoost 분류 모델

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [9]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', 
                    max_depth=6, learning_rate=0.1, n_estimators=100, 
                    random_state=2, n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('점수: ' + str(score))

점수: 0.9736842105263158


`accuracy_score()` 함수 대신 `score()` 메서드를 사용할 수 있습니다.

In [10]:
xgb.score(X_test, y_test)

0.9736842105263158

XGBoost의 기본 파이썬 API를 사용하는 경우 부스터(Booster) 객체의 `predict()` 메서드는 `multi:softprob`일 때 확률을 반환하고 `multi:softmax`일 때 클래스 레이블을 반환합니다.

In [11]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test[:5])

param = {'objective': 'multi:softprob', 'num_class': 3}
bstr = xgb.train(param, dtrain, 10)
bstr.predict(dtest)

array([[0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.02841366, 0.05416913, 0.9174172 ],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142]], dtype=float32)

In [12]:
param = {'objective': 'multi:softmax', 'num_class': 3}
bstr = xgb.train(param, dtrain, 10)
bstr.predict(dtest)

array([0., 0., 2., 0., 0.], dtype=float32)

### 당뇨병 데이터셋

In [13]:
X, y = datasets.load_diabetes(return_X_y=True)

#### XGBoost 회귀 모델 (교차 검증)

In [14]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror', 
                   max_depth=6, learning_rate=0.1, n_estimators=100, 
                   random_state=2, n_jobs=-1)

scores = cross_val_score(xgb, X, y, 
                         scoring='neg_mean_squared_error', cv=5)

# 평가 점수의 제곱근을 계산한합니다.
rmse = np.sqrt(-scores)

# RMSE를 출력합니다.
print('RMSE:', np.round(rmse, 3))

# 평균 점수를 출력합니다.
print('RMSE 평균: %0.3f' % (rmse.mean()))

RMSE: [63.033 59.689 64.538 63.699 64.661]
RMSE 평균: 63.124


In [15]:
pd.DataFrame(y).describe()

,0
count,442.000000
mean,152.133484
std,77.093005
min,25.000000
25%,87.000000
50%,140.500000
75%,211.500000
max,346.000000


## 힉스 보손 찾기 - 사례연구

### 데이터

In [16]:
df = pd.read_csv('atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')
df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label,KaggleSet,KaggleWeight
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2.150,0.444,46.062,1.24,-2.475,113.497,0.000814,s,t,0.002653
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,0.725,1.158,-999.000,-999.00,-999.000,46.226,0.681042,b,t,2.233584
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,2.053,-2.028,-999.000,-999.00,-999.000,44.251,0.715742,b,t,2.347389
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,1.660654,b,t,5.446378
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,1.904263,b,t,6.245333


In [17]:
del df['Weight']
del df['KaggleSet']
df = df.rename(columns={"KaggleWeight": "Weight"})
label_col = df['Label']
del df['Label']
df['Label'] = label_col

##### 데이터 프레임의 열을 삭제하고 옮기는 방법

In [18]:
df_h = pd.read_csv('atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')

In [19]:
df_new = df_h.drop(['Weight', 'KaggleSet', 'Label'], axis=1).assign(Label=df_h['Label'])
df_new = df_new.rename(columns={'KaggleWeight': 'Weight'})

In [20]:
df_new.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [21]:
df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   EventId                      250000 non-null  int64  
 1   DER_mass_MMC                 250000 non-null  float64
 2   DER_mass_transverse_met_lep  250000 non-null  float64
 3   DER_mass_vis                 250000 non-null  float64
 4   DER_pt_h                     250000 non-null  float64
 5   DER_deltaeta_jet_jet         250000 non-null  float64
 6   DER_mass_jet_jet             250000 non-null  float64
 7   DER_prodeta_jet_jet          250000 non-null  float64
 8   DER_deltar_tau_lep           250000 non-null  float64
 9   DER_pt_tot                   250000 non-null  float64
 10  DER_sum_pt                   250000 non-null  float64
 11  DER_pt_ratio_lep_tau         250000 non-null  float64
 12  DER_met_phi_centrality       250000 non-null  float64
 13 

In [23]:
df['Label'].replace(('s', 'b'), (1, 0), inplace=True)

In [24]:
X = df.iloc[:,1:31]
y = df.iloc[:,-1]

##### 열을 선택하는 다른 방법

In [25]:
X_new = df_new.loc[:, ~df_new.columns.isin(['EventId','Weight','Label'])]
y_new = df_new.loc[:, 'Label']

In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_new = le.fit_transform(df_new['Label'])
y_new

array([1, 0, 0, ..., 1, 0, 0])

In [27]:
print(le.classes_)

['b' 's']


### 측정 지표

In [28]:
xgb = XGBClassifier(n_estimators=5, random_state=2)
xgb.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric='mlogloss')

[0]	validation_0-mlogloss:0.73655	validation_1-mlogloss:0.74850
[1]	validation_0-mlogloss:0.52525	validation_1-mlogloss:0.54465
[2]	validation_0-mlogloss:0.38855	validation_1-mlogloss:0.41754
[3]	validation_0-mlogloss:0.29493	validation_1-mlogloss:0.33279
[4]	validation_0-mlogloss:0.22880	validation_1-mlogloss:0.27454


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=5, n_jobs=6,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=2, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [29]:
xgb.evals_result()

{'validation_0': OrderedDict([('mlogloss',
               [0.736555, 0.525247, 0.388554, 0.294931, 0.228803])]),
 'validation_1': OrderedDict([('mlogloss',
               [0.748503, 0.544654, 0.417538, 0.33279, 0.274544])])}

### 가중치

In [30]:
df['test_Weight'] = df['Weight'] * 550000 / len(y)

In [31]:
s = np.sum(df[df['Label']==1]['test_Weight'])
b = np.sum(df[df['Label']==0]['test_Weight'])

In [32]:
b/s

593.9401931492318

### 모델

In [33]:
import xgboost as xgb

# 넘파이 배열로 xgboost.DMatrix 만들기. -999.0는 누락된 값으로 다룹니다.
xgmat = xgb.DMatrix(X, y, missing = -999.0, weight=df['test_Weight'])

# xgboost 파라미터 설정
param = {}
# 랭킹만 필요하기 때문에 로지스틱 함수를 적용하기 전의 출력을 사용합니다.
param['objective'] = 'binary:logitraw'
# 양성 샘플의 가중치를 조정합니다.
param['scale_pos_weight'] = b/s
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'

# ams 지표를 추가합니다.
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [(xgmat,'train')]

# 120개 트리 부스팅
num_round = 120

print('데이터 로딩 완료, 트리 부스팅 시작')
bst = xgb.train(plst, xgmat, num_round, watchlist)
bst.save_model('higgs.model')
print('훈련 종료')

데이터 로딩 완료, 트리 부스팅 시작
[0]	train-auc:0.91088	train-ams@0.15:3.68890
[1]	train-auc:0.91532	train-ams@0.15:3.96782
[2]	train-auc:0.91797	train-ams@0.15:4.09138
[3]	train-auc:0.91950	train-ams@0.15:4.24051
[4]	train-auc:0.92047	train-ams@0.15:4.24758
[5]	train-auc:0.92130	train-ams@0.15:4.25453
[6]	train-auc:0.92223	train-ams@0.15:4.33225
[7]	train-auc:0.92335	train-ams@0.15:4.35882
[8]	train-auc:0.92386	train-ams@0.15:4.37567
[9]	train-auc:0.92424	train-ams@0.15:4.36435
[10]	train-auc:0.92481	train-ams@0.15:4.36440
[11]	train-auc:0.92540	train-ams@0.15:4.41066
[12]	train-auc:0.92581	train-ams@0.15:4.41476
[13]	train-auc:0.92641	train-ams@0.15:4.46029
[14]	train-auc:0.92684	train-ams@0.15:4.43823
[15]	train-auc:0.92734	train-ams@0.15:4.47194
[16]	train-auc:0.92801	train-ams@0.15:4.52107
[17]	train-auc:0.92844	train-ams@0.15:4.57147
[18]	train-auc:0.92898	train-ams@0.15:4.60306
[19]	train-auc:0.92933	train-ams@0.15:4.65702
[20]	train-auc:0.92984	train-ams@0.15:4.69331
[21]	train-auc:0.93013	

##### 사이킷런 API로 구현하기

In [35]:
clf = XGBClassifier(n_estimators=120, learning_rate=0.1, missing=-999.0, 
                    scale_pos_weight=b/s, random_state=2)

clf.fit(X, y, sample_weight=df['test_Weight'], 
        eval_set=[(X, y)], eval_metric=['auc', 'ams@0.15'],
        sample_weight_eval_set=[df['test_Weight']])

clf.save_model('higgs-sklearn.model')

[0]	validation_0-auc:0.91088	validation_0-ams@0.15:3.68916
[1]	validation_0-auc:0.91528	validation_0-ams@0.15:3.96859
[2]	validation_0-auc:0.91771	validation_0-ams@0.15:4.05796
[3]	validation_0-auc:0.91931	validation_0-ams@0.15:4.20578
[4]	validation_0-auc:0.92011	validation_0-ams@0.15:4.14346
[5]	validation_0-auc:0.92099	validation_0-ams@0.15:4.16774
[6]	validation_0-auc:0.92191	validation_0-ams@0.15:4.26030
[7]	validation_0-auc:0.92231	validation_0-ams@0.15:4.26365
[8]	validation_0-auc:0.92330	validation_0-ams@0.15:4.32430
[9]	validation_0-auc:0.92415	validation_0-ams@0.15:4.38294
[10]	validation_0-auc:0.92470	validation_0-ams@0.15:4.38984
[11]	validation_0-auc:0.92528	validation_0-ams@0.15:4.40630
[12]	validation_0-auc:0.92589	validation_0-ams@0.15:4.44715
[13]	validation_0-auc:0.92630	validation_0-ams@0.15:4.45614
[14]	validation_0-auc:0.92692	validation_0-ams@0.15:4.48812
[15]	validation_0-auc:0.92739	validation_0-ams@0.15:4.52030
[16]	validation_0-auc:0.92804	validation_0-ams@0.1

In [36]:
clf.evals_result()

{'validation_0': OrderedDict([('auc',
               [0.910879,
                0.915277,
                0.917712,
                0.919314,
                0.920109,
                0.920992,
                0.921914,
                0.922306,
                0.923298,
                0.924153,
                0.924703,
                0.925281,
                0.925889,
                0.926301,
                0.926918,
                0.927393,
                0.928042,
                0.928428,
                0.928996,
                0.929478,
                0.929999,
                0.930363,
                0.93069,
                0.931093,
                0.931384,
                0.931678,
                0.932015,
                0.932341,
                0.932681,
                0.933036,
                0.933275,
                0.93359,
                0.933943,
                0.934161,
                0.934456,
                0.934762,
                0.935109,
  

In [37]:
clf.score(X, y)

0.800476